In [2]:
import torch
import foolbox as fb
import timm
from PIL import Image
from urllib.request import urlopen
import requests
import numpy as np
import torchvision as tivision

/home/michaelangelo/Benchmark-of-Methods-for-Adversarial-Attacks/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
vgg_model = timm.create_model('vgg16.tv_in1k', pretrained=True)
vgg_model = vgg_model.cuda().eval()


NO LABELS PROVIDED! 
However in docs we can see than this model was trained on ImageNet1k.

In [4]:
url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
imagenet_labels = requests.get(url).json()

In [88]:
def get_label_for_image(image_url: str, model_name: str = 'vgg16.tv_in1k'):
    model = timm.create_model(model_name, pretrained=True)
    model.eval()
    
    img = Image.open(urlopen(image_url))

    image_transforms = tivision.transforms.Compose([
        tivision.transforms.Resize((224,224)),
        tivision.transforms.ToTensor()
    ])


    data_config = timm.data.resolve_model_data_config(model)
    
    transforms = timm.data.create_transform(**data_config, is_training=False)
    input_tensor = image_transforms(img).unsqueeze(0)
    
    with torch.no_grad():
        logits = model(input_tensor)
    
    probabilities = torch.nn.functional.softmax(logits[0], dim=0)
    
    url = "https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json"
    imagenet_labels = requests.get(url).json()
    
    assert len(imagenet_labels) == logits.shape[1], "Mismatch between labels and logits"
    
    predicted_index = torch.argmax(probabilities).item()
    predicted_label = imagenet_labels[predicted_index]
    
    return predicted_label, probabilities[predicted_index].item()



In [89]:

img_list = [
    'https://as2.ftcdn.net/v2/jpg/03/05/26/83/1000_F_305268343_5Xi5esuvd6mIOqFC0QXZdCcqIWNQ6HR2.jpg',
    'https://as2.ftcdn.net/v2/jpg/02/41/23/39/1000_F_241233928_3UtxKpchsTSbR4iJG9j8xSRYh3I8MyzG.jpg'
]

for image_url in img_list:
    label, confidence = get_label_for_image(image_url)
    print(f"Predicted Label: {label} with confidence: {confidence * 100:.2f}%")


Predicted Label: great egret with confidence: 18.57%
Predicted Label: ostrich with confidence: 42.62%


In [15]:
print(torch.cuda.is_available())

True


PGD

In [93]:

def fool_model_with_pgd(image_url: str, model):
    
    # Foolbox model wrapper
    fmodel = fb.PyTorchModel(
        model, 
        bounds=(0, 1), 
    )

    image_transforms = tivision.transforms.Compose([
        tivision.transforms.Resize((224,224)),
        tivision.transforms.ToTensor()
    ])
    
    
    img = Image.open(urlopen(image_url))
    img_tensor = image_transforms(img).unsqueeze(0).cuda()


    np_arr = np.zeros(1)
    np_arr[0] = 123

    label = torch.from_numpy(np_arr).to(dtype=torch.long).cuda()
    criterion= fb.criteria.TargetedMisclassification(label)

    attack = fb.attacks.LinfPGD()
    raw_adversarial, _, success = attack(fmodel, img_tensor, criterion=criterion, epsilons=0.1)
    
    adversarial_label = torch.argmax(fmodel(raw_adversarial)).item()
    
    return raw_adversarial, success, label, adversarial_label


In [94]:
raw_adversarial,  success, label, adversarial_label = fool_model_with_pgd(    'https://as2.ftcdn.net/v2/jpg/03/05/26/83/1000_F_305268343_5Xi5esuvd6mIOqFC0QXZdCcqIWNQ6HR2.jpg', vgg_model)


In [95]:
print(imagenet_labels[adversarial_label])

spiny lobster


JSMA NOT PRESENT IN FOOLBOX

CARLINI

In [15]:
def fool_model_with_cwl2(image_url: str, model):
 
    
    # Foolbox model wrapper
    fmodel = fb.PyTorchModel(
        model, 
        bounds=(0, 1), 
    )

    image_transforms = tivision.transforms.Compose([
        tivision.transforms.Resize((224,224)),
        tivision.transforms.ToTensor()
    ])
    
    img = Image.open(urlopen(image_url))
    img_tensor = image_transforms(img).unsqueeze(0).cuda()


    np_arr = np.zeros(1)
    np_arr[0] = 123

    label = torch.from_numpy(np_arr).to(dtype=torch.long).cuda()
    criterion= fb.criteria.TargetedMisclassification(label)

    attack = fb.attacks.L2CarliniWagnerAttack(binary_search_steps=10,steps = 100, stepsize=0.1, confidence=0.0)
    raw_adversarial, _, success = attack(fmodel, img_tensor, criterion=criterion, epsilons=0.1)
    
    adversarial_label = torch.argmax(fmodel(raw_adversarial)).item()
    
    return raw_adversarial, success, label, adversarial_label


In [17]:
raw_adversarial,  success, label, adversarial_label = fool_model_with_cwl2(    'https://as2.ftcdn.net/v2/jpg/03/05/26/83/1000_F_305268343_5Xi5esuvd6mIOqFC0QXZdCcqIWNQ6HR2.jpg', vgg_model)


In [18]:
print(imagenet_labels[adversarial_label])

spiny lobster


In [19]:
print(adversarial_label)

123


FOOLBOX L2DEEPFOOL

In [20]:
def fool_model_with_deepfool(image_url: str,model, label=9):
    
    # Foolbox model wrapper
    fmodel = fb.PyTorchModel(
        model, 
        bounds=(0, 1), 
    )

    image_transforms = tivision.transforms.Compose([
        tivision.transforms.Resize((224,224)),
        tivision.transforms.ToTensor()
    ])
    
    img = Image.open(urlopen(image_url))
    img_tensor = image_transforms(img).unsqueeze(0).cuda()


    np_arr = np.zeros(1)
    np_arr[0] = label
    
    labels = torch.from_numpy(np_arr).to(dtype=torch.long).cuda()
    criterion= fb.criteria.Misclassification(labels=labels)

    attack = fb.attacks.L2DeepFoolAttack(steps=100, candidates=10, overshoot=0.000001, )
    raw_adversarial, _, success = attack(fmodel, img_tensor, criterion=criterion, epsilons=0.1)
    
    # Check the new adversarial label
    adversarial_label = torch.argmax(fmodel(raw_adversarial)).item()
    
    return raw_adversarial,  success, label, adversarial_label

In [21]:
raw_adversarial, success, label, adversarial_label = fool_model_with_deepfool('https://transforms.stlzoo.org/production/animals/red-kangaroo-02-01.jpg?w=1200&h=1200&auto=compress%2Cformat&fit=crop&dm=1654795233&s=5f137aa9a410a7ea3386c6972265111d',vgg_model ,imagenet_labels.index('dingo'))

In [22]:
print(imagenet_labels[adversarial_label])

hare


EADEN ATTACK

In [71]:
def fool_model_with_eaden(image_url: str,model ,label=9):

    # Foolbox model wrapper
    fmodel = fb.PyTorchModel(
        model, 
        bounds=(0, 1), 
    )

    image_transforms = tivision.transforms.Compose([
        tivision.transforms.Resize((224,224)),
        tivision.transforms.ToTensor()
    ])
    
    img = Image.open(urlopen(image_url))
    img_tensor = image_transforms(img).unsqueeze(0).cuda()


    np_arr = np.zeros(1)
    np_arr[0] = label
    
    labels = torch.from_numpy(np_arr).to(dtype=torch.long).cuda()
    criterion= fb.criteria.Misclassification(labels=labels)

    attack = fb.attacks.EADAttack(binary_search_steps=10, steps=10,decision_rule='EN', )
    raw_adversarial, _, success = attack(fmodel, img_tensor, criterion=criterion, epsilons=0.1)
    
    # Check the new adversarial label
    adversarial_label = torch.argmax(fmodel(raw_adversarial)).item()
    
    return raw_adversarial, success, label, adversarial_label

In [72]:
raw_adversarial, success, label, adversarial_label = fool_model_with_eaden('https://transforms.stlzoo.org/production/animals/red-kangaroo-02-01.jpg?w=1200&h=1200&auto=compress%2Cformat&fit=crop&dm=1654795233&s=5f137aa9a410a7ea3386c6972265111d',vgg_model ,imagenet_labels.index('dingo'))

In [25]:
print(imagenet_labels[adversarial_label])

hare


HOPSKIPJUMP

In [62]:
def fool_model_with_hopskipjump(image_url: str, model ,label=9):
    
    # Foolbox model wrapper
    fmodel = fb.PyTorchModel(
        model, 
        bounds=(0, 1), 
    )

    image_transforms = tivision.transforms.Compose([
        tivision.transforms.Resize((224,224)),
        tivision.transforms.ToTensor()
    ])
    
    img = Image.open(urlopen(image_url))
    img_tensor = image_transforms(img).unsqueeze(0).cuda()


    np_arr = np.zeros(1)
    np_arr[0] = label
    
    labels = torch.from_numpy(np_arr).to(dtype=torch.long).cuda()
    criterion= fb.criteria.Misclassification(labels=labels)

    attack = fb.attacks.HopSkipJumpAttack(steps=10,)
    raw_adversarial, _, success = attack(fmodel, img_tensor, criterion=criterion, epsilons=0.1)
    
    # Check the new adversarial label
    adversarial_label = torch.argmax(fmodel(raw_adversarial)).item()
    
    return raw_adversarial, success, label, adversarial_label

In [63]:
raw_adversarial, success, label, adversarial_label = fool_model_with_hopskipjump('https://transforms.stlzoo.org/production/animals/red-kangaroo-02-01.jpg?w=1200&h=1200&auto=compress%2Cformat&fit=crop&dm=1654795233&s=5f137aa9a410a7ea3386c6972265111d',vgg_model ,imagenet_labels.index('dingo'))

In [64]:
print(imagenet_labels[adversarial_label])

coyote


In [65]:
import time
import pandas as pd

In [96]:
def benchmark_methods(methods: dict, image_url: str, model, num_of_iters=7):
    """
    Benchmark a set of adversarial attack methods.

    Args:
        methods (dict): Dictionary of attack methods {method_name: function}.
        image_url (str): URL of the input image.
        model: Pretrained model to attack.

    Returns:
        pd.DataFrame: Benchmarking results as a tidy table.
    """
    results = []

    data_config = timm.data.resolve_model_data_config(model)
    image_transforms = timm.data.create_transform(**data_config, is_training=False)


    for method_name, method in methods.items():
        times = []
        probabilities_before = []
        probabilities_after = []
        label_before = []
        label_after = []
        # successes = []

        print(f"Benchmarking {method_name}...")

        for _ in range(num_of_iters):  # Run each method 3 times
            start_time = time.time()
            
            # Run the adversarial method
            outputs = method(image_url, model)
            raw_adversarial, _, _, _ = outputs
            
            img = Image.open(urlopen(image_url))
            input_tensor = image_transforms(img).unsqueeze(0).cuda()
            
            
            # Measure time
            elapsed_time = time.time() - start_time
            times.append(elapsed_time)
            
            # Collect probabilities
            with torch.no_grad():
                # For the original input
                logits_before = model(input_tensor)
                probabilities_before_run = torch.nn.functional.softmax(logits_before[0], dim=0)
                probabilities_before.append(torch.max(probabilities_before_run).cpu().detach().numpy())
                label_before.append(int(torch.argmax(logits_before).cpu().detach().numpy()))
                
                

                # For the adversarial input
                logits_after = model(raw_adversarial)
                probabilities_after_run = torch.nn.functional.softmax(logits_after, dim=1)
                probabilities_after.append(torch.max(probabilities_after_run).cpu().detach().numpy())
                label_after.append(int(torch.argmax(logits_after).cpu().detach().numpy()))
                
            
            # Track success
            # successes.append(success)
        
        # Store results for this method
        results.append({
            "Method": method_name,
            "Avg Time (s)": np.mean(times),
            "Avg Prob Before": np.mean(probabilities_before),
            "Avg Prob After": np.mean(probabilities_after),
            "Predicted Label Before": label_before,
            "Predicted Label After": label_after,
            # "Success Rate": np.mean(successes),
        })

    # Convert to DataFrame for a tidy table
    return pd.DataFrame(results)

In [99]:
methods = {
    "CW_L2": fool_model_with_cwl2,
    "PGD": fool_model_with_pgd,
    "HopSkipJump": fool_model_with_hopskipjump,
    "DeepFool": fool_model_with_deepfool,
    "EAD_EN": fool_model_with_eaden,
}


image_url ='https://as2.ftcdn.net/v2/jpg/03/05/26/83/1000_F_305268343_5Xi5esuvd6mIOqFC0QXZdCcqIWNQ6HR2.jpg'  


benchmark_results = benchmark_methods(methods, image_url, vgg_model, num_of_iters=5)

display(benchmark_results)

Benchmarking CW_L2...
Benchmarking PGD...
Benchmarking HopSkipJump...
Benchmarking DeepFool...
Benchmarking EAD_EN...


,Method,Avg Time (s),Avg Prob Before,Avg Prob After,Predicted Label Before,Predicted Label After
0,CW_L2,34.526212,0.998978,0.222788,"[9, 9, 9, 9, 9]","[123, 123, 123, 123, 123]"
1,PGD,5.903388,0.998978,0.999999,"[9, 9, 9, 9, 9]","[123, 123, 123, 123, 123]"
2,HopSkipJump,127.597958,0.998978,0.185621,"[9, 9, 9, 9, 9]","[132, 132, 132, 132, 132]"
3,DeepFool,0.827010,0.998978,0.185621,"[9, 9, 9, 9, 9]","[132, 132, 132, 132, 132]"
4,EAD_EN,19.370939,0.998978,0.185621,"[9, 9, 9, 9, 9]","[132, 132, 132, 132, 132]"
